# 04 - 記憶管理 (Memory Management)

## 📚 學習目標

- ✅ 理解 LangChain 的記憶機制
- ✅ 掌握短期記憶 (ConversationBufferMemory)
- ✅ 學習自訂 State Schema 管理對話狀態
- ✅ 實作帶記憶的客服 Agent
- ✅ 使用 Middleware 增強記憶功能

## ⏰ 預計時間: 45 分鐘

## 📦 環境設定

In [ ]:
!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install langgraph==0.0.20
!pip install python-dotenv==1.0.0

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ 環境設定完成")

## Part 1: 為什麼需要記憶?

### 1.1 無記憶的 Agent

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

# 第一次對話
response1 = llm.invoke("我的保單號碼是 CL202401001")
print("👤 使用者: 我的保單號碼是 CL202401001")
print(f"🤖 AI: {response1.content}\n")

# 第二次對話
response2 = llm.invoke("請查詢我的保單資訊")
print("👤 使用者: 請查詢我的保單資訊")
print(f"🤖 AI: {response2.content}")
print("\n❌ 問題: AI 不記得之前說過的保單號碼!")

### 1.2 手動管理對話歷史

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# 手動維護對話歷史
messages = [
    SystemMessage(content="你是國泰人壽的客服助理"),
    HumanMessage(content="我的保單號碼是 CL202401001"),
]

response1 = llm.invoke(messages)
messages.append(AIMessage(content=response1.content))
print(f"🤖 AI: {response1.content}\n")

# 第二次對話 - 記得之前的內容
messages.append(HumanMessage(content="請查詢我的保單資訊"))
response2 = llm.invoke(messages)
print(f"🤖 AI: {response2.content}")
print("\n✅ AI 記得保單號碼了!")

## Part 2: ConversationBufferMemory

### 2.1 基礎用法

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# 建立記憶
memory = ConversationBufferMemory()

# 建立對話鏈
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # 顯示執行過程
)

# 第一輪對話
response1 = conversation.predict(input="我叫王小明,我的保單號碼是 CL202401001")
print(f"\n回應: {response1}")

# 第二輪對話
response2 = conversation.predict(input="請問我剛才說的保單號碼是什麼?")
print(f"\n回應: {response2}")

# 第三輪對話
response3 = conversation.predict(input="我叫什麼名字?")
print(f"\n回應: {response3}")

### 2.2 查看記憶內容

In [ ]:
# 查看完整對話歷史
print("💾 對話歷史:")
print(memory.buffer)

# 查看結構化的訊息列表
print("\n📋 訊息列表:")
messages = memory.chat_memory.messages
for i, msg in enumerate(messages, 1):
    role = "👤" if msg.type == "human" else "🤖"
    print(f"{i}. {role} {msg.content}")

### 2.3 清除記憶

In [ ]:
# 清除記憶
memory.clear()
print("🗑️ 記憶已清除")

# 測試
response = conversation.predict(input="我叫什麼名字?")
print(f"\n回應: {response}")
print("\n👆 AI 已經不記得之前的對話了")

## Part 3: 自訂 State Schema

### 3.1 使用 TypedDict 定義狀態

In [ ]:
from typing import TypedDict, Annotated, List
from operator import add
from langgraph.graph import StateGraph, END

class ConversationState(TypedDict):
    """對話狀態"""
    # 對話歷史 (累加)
    messages: Annotated[List[dict], add]
    
    # 使用者資訊 (覆蓋)
    user_name: str
    policy_id: str
    
    # 當前意圖 (覆蓋)
    current_intent: str
    
    # 執行日誌 (累加)
    execution_log: Annotated[List[str], add]

print("✅ 自訂狀態結構定義完成")

### 3.2 建立帶狀態的 Agent

In [ ]:
from datetime import datetime

def extract_info(state: ConversationState) -> ConversationState:
    """擷取使用者資訊"""
    last_msg = state["messages"][-1]["content"] if state["messages"] else ""
    
    # 簡單的資訊擷取 (實際會用 LLM)
    user_name = state.get("user_name", "")
    policy_id = state.get("policy_id", "")
    
    if "我叫" in last_msg or "我是" in last_msg:
        # 提取姓名
        for word in ["我叫", "我是"]:
            if word in last_msg:
                user_name = last_msg.split(word)[1].split(",")[0].split("。")[0].strip()
                break
    
    if "保單" in last_msg and "CL" in last_msg:
        # 提取保單號碼
        import re
        match = re.search(r'CL\d+', last_msg)
        if match:
            policy_id = match.group()
    
    return {
        "user_name": user_name,
        "policy_id": policy_id,
        "execution_log": [f"[{datetime.now().strftime('%H:%M:%S')}] 擷取資訊完成"]
    }

def generate_response(state: ConversationState) -> ConversationState:
    """生成回應"""
    last_msg = state["messages"][-1]["content"]
    user_name = state.get("user_name", "")
    policy_id = state.get("policy_id", "")
    
    # 建立提示詞
    context = f"""
已知資訊:
- 使用者姓名: {user_name if user_name else '未知'}
- 保單號碼: {policy_id if policy_id else '未知'}

使用者問題: {last_msg}

請根據已知資訊回答問題。如果資訊不足,請友善地詢問。
"""
    
    response = llm.invoke(context)
    
    return {
        "messages": [{"role": "assistant", "content": response.content}],
        "execution_log": [f"[{datetime.now().strftime('%H:%M:%S')}] 生成回應完成"]
    }

# 建立 StateGraph
workflow = StateGraph(ConversationState)
workflow.add_node("extract", extract_info)
workflow.add_node("respond", generate_response)

workflow.set_entry_point("extract")
workflow.add_edge("extract", "respond")
workflow.add_edge("respond", END)

stateful_agent = workflow.compile()

print("✅ 帶狀態的 Agent 建立完成")

### 3.3 測試狀態管理

In [ ]:
# 初始化狀態
state = {
    "messages": [{"role": "user", "content": "我叫王小明,我的保單是 CL202401001"}],
    "user_name": "",
    "policy_id": "",
    "current_intent": "",
    "execution_log": []
}

# 第一輪對話
result1 = stateful_agent.invoke(state)

print("💬 第一輪對話:")
print(f"👤 使用者: {result1['messages'][0]['content']}")
print(f"🤖 AI: {result1['messages'][1]['content']}")
print(f"\n📊 狀態:")
print(f"  姓名: {result1['user_name']}")
print(f"  保單: {result1['policy_id']}")
print(f"\n📝 日誌:")
for log in result1['execution_log']:
    print(f"  {log}")

In [ ]:
# 第二輪對話 - 使用第一輪的狀態
result1["messages"].append({"role": "user", "content": "請問我的保單號碼是什麼?"})

result2 = stateful_agent.invoke(result1)

print("\n💬 第二輪對話:")
print(f"👤 使用者: {result2['messages'][-2]['content']}")
print(f"🤖 AI: {result2['messages'][-1]['content']}")
print(f"\n✅ AI 記得之前說過的保單號碼: {result2['policy_id']}")

## Part 4: Middleware 增強記憶

### 4.1 建立記憶中介層

In [ ]:
class MemoryMiddleware:
    """記憶中介層"""
    
    def __init__(self):
        self.user_data = {}  # 使用者資料快取
        self.conversation_count = {}  # 對話計數
    
    def before_invoke(self, state: ConversationState) -> ConversationState:
        """執行前處理"""
        user_name = state.get("user_name", "")
        
        if user_name:
            # 從快取載入使用者資料
            if user_name in self.user_data:
                print(f"📂 從快取載入 {user_name} 的資料")
                state.update(self.user_data[user_name])
            
            # 更新對話計數
            self.conversation_count[user_name] = self.conversation_count.get(user_name, 0) + 1
        
        return state
    
    def after_invoke(self, state: ConversationState) -> ConversationState:
        """執行後處理"""
        user_name = state.get("user_name", "")
        
        if user_name:
            # 儲存使用者資料到快取
            self.user_data[user_name] = {
                "user_name": user_name,
                "policy_id": state.get("policy_id", ""),
                "conversation_count": self.conversation_count.get(user_name, 0)
            }
            print(f"💾 儲存 {user_name} 的資料到快取")
        
        return state
    
    def get_user_stats(self, user_name: str) -> dict:
        """取得使用者統計"""
        return self.user_data.get(user_name, {})

# 建立中介層實例
memory_middleware = MemoryMiddleware()

print("✅ 記憶中介層建立完成")

### 4.2 測試 Middleware

In [ ]:
# 模擬第一次對話
state1 = {
    "messages": [{"role": "user", "content": "我叫李小華,保單 CL202401002"}],
    "user_name": "李小華",
    "policy_id": "CL202401002",
    "current_intent": "",
    "execution_log": []
}

print("🔄 第一次對話:")
state1 = memory_middleware.before_invoke(state1)
# ... 執行 Agent ...
state1 = memory_middleware.after_invoke(state1)

# 模擬第二次對話 (新會話)
state2 = {
    "messages": [{"role": "user", "content": "我想查詢保單"}],
    "user_name": "李小華",  # 同一個使用者
    "policy_id": "",  # 沒有提供保單號碼
    "current_intent": "",
    "execution_log": []
}

print("\n🔄 第二次對話 (同一使用者):")
state2 = memory_middleware.before_invoke(state2)
print(f"\n✅ 自動填入保單號碼: {state2['policy_id']}")
print(f"✅ 對話次數: {memory_middleware.conversation_count['李小華']}")

# 查看使用者統計
print("\n📊 使用者統計:")
stats = memory_middleware.get_user_stats("李小華")
print(stats)

## Part 5: 完整客服系統

### 5.1 整合所有功能

In [ ]:
class CustomerServiceState(TypedDict):
    """客服系統狀態"""
    session_id: str
    messages: Annotated[List[dict], add]
    user_name: str
    policy_id: str
    intent: str
    sentiment: str  # positive/neutral/negative
    resolved: bool
    execution_log: Annotated[List[str], add]

def analyze_message(state: CustomerServiceState) -> CustomerServiceState:
    """分析訊息意圖和情感"""
    last_msg = state["messages"][-1]["content"]
    
    # 使用 LLM 分析
    analysis_prompt = f"""
分析以下客戶訊息:
{last_msg}

請判斷:
1. 意圖 (查詢/投訴/購買/其他)
2. 情感 (positive/neutral/negative)

只回答 JSON: {{"intent": "...", "sentiment": "..."}}
"""
    
    response = llm.invoke(analysis_prompt)
    import json
    try:
        analysis = json.loads(response.content)
    except:
        analysis = {"intent": "其他", "sentiment": "neutral"}
    
    return {
        "intent": analysis["intent"],
        "sentiment": analysis["sentiment"],
        "execution_log": [f"[分析] 意圖: {analysis['intent']}, 情感: {analysis['sentiment']}"]
    }

def handle_query(state: CustomerServiceState) -> CustomerServiceState:
    """處理客戶查詢"""
    context = f"""
客戶資訊:
- 姓名: {state.get('user_name', '未知')}
- 保單: {state.get('policy_id', '未知')}
- 意圖: {state.get('intent', '未知')}
- 情感: {state.get('sentiment', '未知')}

對話歷史:
"""
    for msg in state["messages"]:
        role = "客戶" if msg["role"] == "user" else "客服"
        context += f"{role}: {msg['content']}\n"
    
    context += "\n請根據以上資訊,提供專業且友善的回應。"
    
    response = llm.invoke(context)
    
    return {
        "messages": [{"role": "assistant", "content": response.content}],
        "resolved": True,  # 簡化處理
        "execution_log": [f"[回應] 已生成客服回應"]
    }

# 建立完整客服 Agent
cs_workflow = StateGraph(CustomerServiceState)
cs_workflow.add_node("analyze", analyze_message)
cs_workflow.add_node("handle", handle_query)

cs_workflow.set_entry_point("analyze")
cs_workflow.add_edge("analyze", "handle")
cs_workflow.add_edge("handle", END)

customer_service = cs_workflow.compile()

print("✅ 完整客服系統建立完成")

### 5.2 測試客服系統

In [ ]:
# 場景 1: 正常查詢
session1 = {
    "session_id": "S001",
    "messages": [{"role": "user", "content": "你好,我想查詢我的保單資訊,保單號碼是 CL202401001"}],
    "user_name": "王小明",
    "policy_id": "CL202401001",
    "intent": "",
    "sentiment": "",
    "resolved": False,
    "execution_log": []
}

result = customer_service.invoke(session1)

print("📞 客服對話:")
print("="*60)
print(f"👤 客戶: {result['messages'][0]['content']}")
print(f"\n🤖 客服: {result['messages'][1]['content']}")
print(f"\n📊 分析結果:")
print(f"  意圖: {result['intent']}")
print(f"  情感: {result['sentiment']}")
print(f"  已解決: {result['resolved']}")
print(f"\n📝 執行日誌:")
for log in result['execution_log']:
    print(f"  {log}")

## 🎯 練習題

### 練習 1: 實作對話摘要功能

當對話超過 10 輪時,自動產生摘要並壓縮歷史記錄

In [ ]:
# TODO: 實作 ConversationSummaryMemory

### 練習 2: 多使用者記憶隔離

實作按 user_id 區分不同使用者的記憶

In [ ]:
# TODO: 實作 MultiUserMemory

## 📝 學習檢查清單

- [ ] 理解為什麼需要記憶管理
- [ ] 使用 ConversationBufferMemory
- [ ] 自訂 State Schema 管理複雜狀態
- [ ] 使用 Annotated 實現累加更新
- [ ] 建立 Middleware 增強記憶功能
- [ ] 整合記憶到客服系統
- [ ] 實作對話歷史追蹤
- [ ] 處理使用者資訊快取

## 下一步
繼續前往 `05-middleware-advanced.ipynb` 學習進階中介層技術 👉